# Transform

In [39]:
import numpy as np
import pandas as pd
import pywsra
import xarray as xr

from configure import read_stored_variable

## Setup

In [40]:
%run 'nb0-datasets.ipynb'
%run -i configure.py

earl_ds = read_stored_variable('earl_ds')
fiona_ds = read_stored_variable('fiona_ds')
ian_ds = read_stored_variable('ian_ds')
# julia_ds = read_stored_variable('julia_ds')
idalia_ds = read_stored_variable('idalia_ds')
lee_ds = read_stored_variable('lee_ds')

/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/opt/homebrew/Caskroom/miniconda/base/envs/wsra-mss/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [46]:
lee_ds

<xarray.Dataset>
Dimensions:                                (time: 2334, wavenumber_east: 65,
                                            wavenumber_north: 65, obs: 5,
                                            wavelength: 8)
Coordinates:
  * wavenumber_east                        (wavenumber_east) float32 -0.07854...
  * wavenumber_north                       (wavenumber_north) float32 -0.0785...
  * wavelength                             (wavelength) float32 366.0 ... 91.0
  * obs                                    (obs) timedelta64[ns] -1 days +23:...
  * time                                   (time) datetime64[ns] 2023-09-08T0...
Data variables: (12/34)
    directional_wave_spectrum              (time, wavenumber_east, wavenumber_north) float32 ...
    directional_wave_spectrum_180          (time, wavenumber_east, wavenumber_north) float32 ...
    dominant_to_secondary_partition_angle  (time) float32 270.9 278.6 ... 41.48
    dominant_wave_direction                (time) float32 -76.19 ... 24.44
    dominant_wave_height                   (time) float32 1.335 1.695 ... 5.813
    dominant_wave_wavelength               (time) float32 272.4 122.7 ... 328.7
    ...                                     ...
    met_sfmr_10m_wind_speed                (time) float32 6.2 5.8 ... 10.3 10.1
    met_sfmr_10m_wind_speed_error          (time) float32 0.02 0.02 ... 0.01
    met_sfmr_rain_rate                     (time) float32 0.0 0.0 ... 0.0 0.0
    met_sfmr_data_validity                 (time) float32 0.0 0.0 ... 0.0 0.0
    met_longitude                          (time) float32 -63.86 ... -69.36
    met_latitude                           (time) float32 17.48 17.49 ... 36.39
Attributes:
    title:                ['WSRA level 4 data']
    history:              ['revision: 1.6; date: 20180816; author: NMS/IPS']
    flight_id:            ['20230908I1', '20230909I1', '20230910I1', '2023091...
    mission_id:           ['0313A LEE', '0713A LEE', '1113A LEE', '1413A LEE'...
    storm_id:             AL132023
    date_created:         ['2023-11-10T20:00:44Z', '2023-11-13T14:28:34Z', '2...
    time_coverage_start:  2023-09-08 08:24:40+00:00
    time_coverage_end:    2023-09-16 14:16:49+00:00
    storm_name:           lee

## WSRA

### Quality control metrics

Compute the standard deviation of the mean square slope observations.  These (5) observations are independent measures of mean square slope offset by -20, -10, 0, +10, and +20 seconds from the reported time.

In [3]:
def mean_square_slope_std(wsra_ds):
    return wsra_ds['sea_surface_mean_square_slope'].std(axis=1)

In [4]:
earl_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(earl_ds)
fiona_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(fiona_ds)
ian_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(ian_ds)
# julia_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(julia_ds)
idalia_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(idalia_ds)
lee_ds['sea_surface_mean_square_slope_std'] = mean_square_slope_std(lee_ds)

### Masking

Mask the WSRA observations based on flight metadata and quality control metrics.

In [21]:
def mask_wsra(wsra_ds, mask_dict):
    wsra_masked_ds = (wsra_ds
                      .wsra.create_trajectory_mask(mask_dict)
                      .wsra.mask())
    num_masked_values = wsra_ds['time_mask'].attrs['num_masked_values']
    perc_masked_values = 100 * num_masked_values / wsra_ds['time_mask'].size
    print(
        f"{wsra_ds.attrs['storm_id']}: "
        f"{num_masked_values} masked values ({perc_masked_values.round(1)}%)."
    )
    return wsra_masked_ds


In [32]:
mask_dict = {
    'wsra_computed_roll': (-2.5, 2.5),
    'platform_radar_altitude': (1000, 4000),
    # 'peak_spectral_variance': (),
    'platform_speed_wrt_ground': (80, 250),
    'met_sfmr_rain_rate': (0, 15),
    'rainfall_rate_median': (0, 15),
    'sea_surface_mean_square_slope_std': (0, 0.1),
}

In [33]:
earl_masked_ds = mask_wsra(earl_ds, mask_dict)
fiona_masked_ds = mask_wsra(fiona_ds, mask_dict)
ian_masked_ds = mask_wsra(ian_ds, mask_dict)
# julia_masked_ds = mask_wsra(julia_ds, mask_dict)
idalia_masked_ds = mask_wsra(idalia_ds, mask_dict)
lee_masked_ds = mask_wsra(lee_ds, mask_dict)

AL062022: 1858 masked values (65.7%).
AL072022: 397 masked values (27.4%).
AL092022: 508 masked values (35.3%).
AL092022: 230 masked values (51.1%).
AL102023: 137 masked values (17.5%).
AL132023: 978 masked values (41.9%).


### DataFrames

In [36]:
def wsra_mss_ds_to_df(wsra_ds, storm_name):
    wsra_df = pd.DataFrame({
        'wind_speed': wsra_ds['met_sfmr_10m_wind_speed'],
        'mss': wsra_ds['sea_surface_mean_square_slope_median'],
        'wind_speed_error': wsra_ds['met_sfmr_10m_wind_speed_error'],
        'wind_speed_validity': wsra_ds['met_sfmr_data_validity'],
        'storm_name': storm_name,
    })
    wsra_df = (wsra_df
               .sort_values(by='wind_speed')
               .dropna()
               .query('0.01 < mss < 0.08')
               .query('0 < wind_speed < 50')
               .query('wind_speed_validity == 0')
               .query('wind_speed_error < 1')
               .reset_index(drop=True))
    return wsra_df

In [37]:
earl_df = wsra_mss_ds_to_df(earl_masked_ds, 'Earl')
fiona_df = wsra_mss_ds_to_df(fiona_masked_ds, 'Fiona')
ian_df = wsra_mss_ds_to_df(ian_masked_ds, 'Ian')
# julia_df = wsra_mss_ds_to_df(julia_masked_ds, 'Julia')
idalia_df = wsra_mss_ds_to_df(idalia_masked_ds, 'Idalia')
lee_df = wsra_mss_ds_to_df(lee_masked_ds, 'Lee')

all_wsra_df = pd.concat(
    [earl_df, fiona_df, ian_df, idalia_df, lee_df]  # julia_df removed
).set_index('storm_name')

storm_names = ['Earl', 'Fiona', 'Ian', 'Idalia', 'Lee']  #  'Julia' removed

### Store

In [38]:
%%capture

%store earl_masked_ds
%store fiona_masked_ds
%store ian_masked_ds
# %store julia_masked_ds
%store idalia_masked_ds
%store lee_masked_ds

%store all_wsra_df